In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
#读训练集、测试集
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
pd.set_option('display.max_columns',10)
#-------------------------------数据预处理------------------------------------

#拷贝原始数据
# 这样即使我们必须对这些数据集进行任何更改，我们也不会丢失原始数据集

train_original = train.copy()
test_original = test.copy()

In [ ]:

#  3+ 替换为3
train['Dependents'].replace('3+', 3, inplace=True)
test['Dependents'].replace('3+', 3, inplace=True)

# y=1,n=0
train['Loan_Status'].replace('N', 0, inplace=True)
train['Loan_Status'].replace('Y', 1, inplace=True)

# 检查下
print(train.head())

In [ ]:

#--------------------------------数据预处理------------------------------------
#----------------------------------缺失值插补-------------------------
print(train.isnull().sum())

# mode 填补
train['Gender'].fillna(train['Gender'].mode()[0], inplace=True)
train['Married'].fillna(train['Married'].mode()[0], inplace=True)
train['Dependents'].fillna(train['Dependents'].mode()[0], inplace=True)
train['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace=True)
train['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace=True)
train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True)
# 连续型用中位数填补
train['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)
# 看看
print(train.isnull().sum())
# replace missing values in Test set with mode/median from Training set
test['Gender'].fillna(train['Gender'].mode()[0], inplace=True)
test['Dependents'].fillna(train['Dependents'].mode()[0], inplace=True)
test['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace=True)
test['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace=True)
test['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True)
test['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)

# log变换
train['LoanAmount_log'] = np.log(train['LoanAmount'])
test['LoanAmount_log'] = np.log(test['LoanAmount'])
# after log transformation

ax1 = plt.subplot(121)
train['LoanAmount_log'].hist(bins=20, figsize=(12,4))
ax1.set_title("Train")

ax2 = plt.subplot(122)
test['LoanAmount_log'].hist(bins=20)
ax2.set_title("Test")
plt.show()


In [ ]:

# 删除 Loan_ID 没啥用
train = train.drop('Loan_ID', axis=1)
test = test.drop('Loan_ID', axis=1)
# Sklearn 需要单独数据集中的目标变量。 因此，从训练数据集中删除目标变量并将其保存在另一个数据集中。
X = train.drop('Loan_Status', 1)
y = train.Loan_Status
# adding dummies to the dataset
X = pd.get_dummies(X)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(X.shape, train.shape, test.shape)
print(X.head())

from sklearn.model_selection import train_test_split
# split the data into train and cross validation set
x_train, x_cv, y_train, y_cv = train_test_split(X, y, test_size=0.3, random_state=0)


In [ ]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
mean_accuracy = []
i=1
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
for train_index,test_index in kf.split(X,y):
      print('\n{} of kfold {}'.format(i,kf.n_splits))
      xtr,xvl = X.loc[train_index],X.loc[test_index]
      ytr,yvl = y[train_index],y[test_index]

      model = CatBoostClassifier(learning_rate=0.03)
      model.fit(xtr, ytr)
      pred_test = model.predict(xvl)
      score = accuracy_score(yvl,pred_test)
      mean_accuracy.append(score)
      print('accuracy_score',score)
      i+=1

print("\n CatBoost Mean validation accuracy: ", sum(mean_accuracy)/len(mean_accuracy))

In [ ]:
x_train, x_cv, y_train, y_cv = train_test_split(X, y, test_size =0.3, random_state=1)
# grid_search2.fit(x_train, y_train)

model = CatBoostClassifier()

grid = {'learning_rate': [0.001,0.005,0.01,0.03],
        'depth': [4, 6, 8,10],
        'l2_leaf_reg': [1, 3],
        'iterations':[400,600,800,1000,2000]}

randomized_search_result = model.randomized_search(grid,
                                                   X=x_train,
                                                   y=y_train,
                                                   plot=True)


网格搜索

In [ ]:
model = CatBoostClassifier(loss_function='Logloss')

grid = {'learning_rate': [0.001,0.005,0.01,0.03],
        'depth': [4, 6, 8,10],
        'l2_leaf_reg': [1, 3],
        'iterations':[400,600,800,1000,2000]}


grid_search_result = model.grid_search(grid,
                                       X=x_train,
                                       y=y_train,
                                       plot=True)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
mean_accuracy = []
i=1
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
for train_index,test_index in kf.split(X,y):
      print('\n{} of kfold {}'.format(i,kf.n_splits))
      xtr,xvl = X.loc[train_index],X.loc[test_index]
      ytr,yvl = y[train_index],y[test_index]

      model = CatBoostClassifier(iterations=1000,learning_rate=0.001,depth=10,l2_leaf_reg=1)
     
      model.fit(xtr, ytr)
      pred_test = model.predict(xvl)
      score = accuracy_score(yvl,pred_test)
      mean_accuracy.append(score)
      print('accuracy_score',score)
      i+=1

print("\n CatBoost Mean validation accuracy: ", sum(mean_accuracy)/len(mean_accuracy))

In [ ]:
cat=CatBoostClassifier(iterations=1000,learning_rate=0.001,l2_leaf_reg=1,depth=10)
cat.fit(x_train,y_train)
fea_ = cat.feature_importances_
fea_name = cat.feature_names_
#sns.barplot(cat.feature_importances_,)
print(fea_)


In [ ]:
print(fea_name)

In [ ]:

plt.figure(figsize=(16, 9))
plt.title('catboost算法计算出的与贷款相关特征重要性')
plt.barh(fea_name,fea_,height =0.5)

In [ ]:
model = CatBoostClassifier(loss_function='Logloss',task_type='GPU')

grid = {'learning_rate': [0.001,0.005,0.01,0.03],
        'depth': [4, 6, 8,10],
        'l2_leaf_reg': [1, 3],
        'iterations':[400,600,800,1000,2000]}


grid_search_result = model.grid_search(grid,
                                       X=x_train,
                                       y=y_train,
                                       plot=True
                                       )

In [ ]:
grid_search_result

In [ ]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
mean_accuracy = []
i=1
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
for train_index,test_index in kf.split(X,y):
      print('\n{} of kfold {}'.format(i,kf.n_splits))
      xtr,xvl = X.loc[train_index],X.loc[test_index]
      ytr,yvl = y[train_index],y[test_index]

      model = CatBoostClassifier(iterations=400,learning_rate=0.01,depth=10,l2_leaf_reg=3)
     
      model.fit(xtr, ytr)
      pred_test = model.predict(xvl)
      score = accuracy_score(yvl,pred_test)
      mean_accuracy.append(score)
      print('accuracy_score',score)
      i+=1

print("\n CatBoost Mean validation accuracy: ", sum(mean_accuracy)/len(mean_accuracy))